<a href="https://colab.research.google.com/github/s21sm/Python/blob/master/d2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# this program is intended to make experiment on Device to Device communication based
# Radio Frequency Fingerprinting positioning system using Received signal strength

import io
from scipy.spatial import distance
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt


In [5]:
# file uploading
uploaded = files.upload()

Saving Training_rss_21Aug17.csv to Training_rss_21Aug17.csv


In [0]:
# csv file reading
train_rss = np.genfromtxt ('Training_rss_21Aug17.csv', delimiter=",")
test_rss = np.genfromtxt ('Test_rss_21Aug17.csv', delimiter=",")
train_coordinate = np.genfromtxt ('Training_coordinates_21Aug17.csv', delimiter=",")
test_coordinate = np.genfromtxt ('Test_coordinates_21Aug17.csv', delimiter=",")

In [0]:
# Defining functions


def estimated_position(train_coordinate, train_rss, test_signature_rss ):
  
 
  estimated_error = 10000000 # initially a bigger value
  
  for i in range(train_rss.shape[0]):
    
   ts = train_rss[i]
   dist = distance.euclidean(ts,test_signature_rss)
   if dist < estimated_error:
    estimated_error = dist
    candidate_train_signature_index = i
    estimated_coordinate=  train_coordinate[candidate_train_signature_index]
   
  return estimated_coordinate

def dictionary_creation(own_device_indx,train_coordinate,train_rss,test_rss,distance_wise_other_device):
    new_dictionary = {}
    multiple_line_answer = []
    for i in range(len(own_device_indx)):
     esti_esti_distance = distance.euclidean(estimated_position(train_coordinate, train_rss, test_rss[own_device_indx[i],:]), estimated_position(train_coordinate, train_rss, test_rss[int(distance_wise_other_device[0][i]),:]))
     D2D_distance = distance.euclidean(test_coordinate[own_device_indx[i],:], test_coordinate[int(distance_wise_other_device[0][i]),:])
     indicator = abs(esti_esti_distance-D2D_distance)
     current_error = distance.euclidean(test_coordinate[own_device_indx[i],:],estimated_position(train_coordinate, train_rss, test_rss[own_device_indx[i],:]) )
     new_dictionary[indicator] = current_error
    return new_dictionary
  
def assign_threshold(): # how far other device should be

 min_t = 10 
 max_t = 15

 return min_t, max_t

def previous_error(train_coordinate, train_rss, test_signature_rss, test_signature_coordinate): # positioning error from tradional method

    estimated = estimated_position(train_coordinate, train_rss, test_signature_rss)
    real = test_signature_coordinate
    error_amount = distance.euclidean(estimated,real)

    return error_amount


def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob


In [0]:
# For training data : changing the initial RSS value 100 to -103
for i in range(train_rss.shape[0]): 
 for j in range(train_rss.shape[1]):
  if train_rss[i,j] == 100:
    train_rss[i,j] = -103
  
  
  


In [0]:
# For test data : changing the initial RSS value 100 to -103
for i in range(test_rss.shape[0]): 
 for j in range(test_rss.shape[1]):
  if test_rss[i,j] == 100:
    test_rss[i,j] = -103

In [0]:
# Main part of the program

final_result_collection = []
correct_n_value =[]

min_threshold = 0.1
max_threshold = 1

for i in range(test_coordinate.shape[0]):  #test_coordinate.shape[0]
 
  #test_signature = test_rss[i,:]
  test_signature_coordinate = test_coordinate[i,:]
  test_signature_rss = test_rss[i,:]
  distance_wise_other_device =np.array([])
  distance_box =np.array([])

  for j in range(test_coordinate.shape[0]):
    coordinate = test_coordinate[j,:]
    dist = distance.euclidean(test_signature_coordinate, coordinate)
    distance_box= np.append(distance_box,dist)

  own_device_indx = np.where((distance_box > min_threshold) & (distance_box < max_threshold))[0]

  if own_device_indx.size >= 3 : 
  
   min_threshold2 = assign_threshold()[0]
   max_threshold2 = assign_threshold()[1]
   other_device_indx = np.where((distance_box > min_threshold2) & (distance_box < max_threshold2))[0]
   other_device_distance = distance_box[other_device_indx]
   sorted_other_device_ditance_index = np.argsort(other_device_distance, axis=0)
   distance_wise_other_device = np.vstack((other_device_indx[sorted_other_device_ditance_index],other_device_distance[sorted_other_device_ditance_index]))   

   if  distance_wise_other_device.shape[1] >= 3:
     new_dictionary = dictionary_creation(own_device_indx,train_coordinate,train_rss,test_rss,distance_wise_other_device) 
     if min(new_dictionary.keys())<=30:
    
       updated_error =new_dictionary[min(new_dictionary.keys())] 
       old_error = previous_error(train_coordinate, train_rss, test_signature_rss, test_signature_coordinate)
       after_before_result = [updated_error,old_error]
       final_result_collection.append(after_before_result)
       correct_n_value.append(i)
       print(i)


  else:
    other_device_indx=np.array([])
    print('empty')




In [0]:

new_collection=[]
old_collection=[]

for i in range(len(final_result_collection)):
  a = final_result_collection[i][0]
  b = final_result_collection[i][1]
  new_collection.append(a)
  old_collection.append(b)

new_collection = np.sort(np.array(new_collection))
old_collection = np.sort(np.array(old_collection)) 


In [0]:
# to plot ECDF of error 

qe, pe = ecdf(new_collection)
plt.plot(qe, pe, '-r', lw=2, label='D2D')

qe1, pe1 = ecdf(old_collection)
plt.plot(qe1, pe1, '-b', lw=2, label='Traditional')

plt.margins(0.00)
plt.axis([0, 15, 0, 1])
plt.yticks([0.68, 0.95])
plt.xticks([5, 10, 15])
plt.xlabel('3D positioning error[m]', fontsize=18)
plt.ylabel('Percentile', fontsize=18)
plt.legend(loc='lower right')

plt.show()

#plt.savefig("trial_fig.png")




In [0]:
# If file need to download from google colab
files.download('trial_fig.png') 